# Transform XML into TEI


In [1]:
import re
import os
from lxml import etree

[x] <article ID=""> : <entry xml:id="">
[x] <Nat_Art>Trévoux 1743</Nat_Art> : removed
[x] <G><vedette>ABAISER,</vedette></G> : <form type="lemma"><orth>ABAISER</orth></form><pc>,</pc> # sortir la virgule ou le point
[x] <svedet> : <form><orth>
[x] <cat_gra> : <gramGrp>
[x] {x} : <pb n="x"/>
[x] <G> : removed
[x] <S> : removed
[x] <I> : removed
[x] <auteur> : removed
[x] <cit> : removed
[x] <oeuvre> : removed
[x] <REM> : <g ref="#manicule-glyph"/>
[x] <dom>termes de Fauconnerie</dom> : <usg type="domain" ana="#fauconnerie">termes de Fauconnerie</usg>
[] <usg type="domain" >termes de Fauconnerie</usg>: ajouter l'attribut <usg type="domain" ana="#fauconnerie">termes de Fauconnerie</usg>
[x] ajout des balises <sense> : après la vedette et à chaque retour chariot à l'intérieur d'une entry
[x]<form type="lemmaGrp"> : intègre le <form type="lemma"> et le <gramGrp>
[] <tab> : ?

In [2]:
def add_sense(content):
    # read file line by line, search for <entry>, then for lines not starting with an element until line starting with </entry>
    # add <sense> before the first line not starting with an element
    # and </sense> before the line starting with </entry>
    lines = content.split('\n')
    new_content = []
    inside_entry = False
    inside_tab = False
    for line in lines:
        stripped_line = line.strip()
        if stripped_line.startswith('<tab>'):
            inside_tab = True
        elif stripped_line.startswith('</tab>'):
            inside_tab = False
            new_content.append(line)
            continue
        if stripped_line.startswith('<entry'):
            inside_entry = True
            new_content.append(line)
        elif inside_entry and stripped_line.startswith('</entry'):
            inside_entry = False
            new_content.append(line)
        elif inside_entry:
            if not inside_tab and not (stripped_line.startswith('<') and stripped_line.endswith('>')) and not '<form type="lemma"' in stripped_line:
                new_content.append('<sense>')
                new_content.append(line)
                new_content.append('</sense>')
            else:
                new_content.append(line)
        else:
            new_content.append(line)
    return '\n'.join(new_content)

def add_lemmaGrp(content):
    lines = content.split('\n')
    new_content = []
    inside_entry = False
    for line in lines:
        stripped_line = line.strip()
        if stripped_line.startswith('<entry'):
            inside_entry = True
            new_content.append(line)
            new_content.append('<form type="lemmaGrp">')
        elif inside_entry and stripped_line.startswith('</entry'):
            inside_entry = False
            new_content.append(line)
        elif inside_entry:
            if '<form type="lemma"' in stripped_line or stripped_line.startswith('<orth>') or stripped_line.startswith('<pb ') or stripped_line.startswith('</form>') or stripped_line.startswith('<pc>') or stripped_line.startswith('<g ref'):
                new_content.append(line)    
            else:
                if stripped_line.startswith('<gramGrp>'):
                    line = line.replace('</gramGrp>', '</gramGrp></form>\n', 1)
                    new_content.append(line)
                else:
                    new_content.append('</form>')
                    new_content.append(line)
                inside_entry = False
        else:
            new_content.append(line)
    return '\n'.join(new_content)



def xml_2_tei(content):

    content = content.replace('\n<Nat_Art>Trévoux 1743</Nat_Art>', '')
    content = content.replace('<article ID=', '<entry id=').replace('</article>', '</entry>')
    content = content.replace('<cat_gra>', '<gramGrp>').replace('</cat_gra>', '</gramGrp>')

    # vedette
    content = content.replace(',</vedette>', '</vedette><pc>,</pc>\n').replace('.</vedette>', '</vedette><pc>.</pc>\n')
    content = content.replace('<vedette>', '<form type="lemma">\n<orth>').replace('</vedette>', '</orth>\n</form>\n')
    
    # page break : problème fin de colonne et pas de page...
    content = re.sub(r'{([0-9]+)}', r'<pb n="\1"/>', content)
    # find: <pb n="146"/> dans TR1.tei
    # déplacer les page break, par exemple ceux qui sont directement dans <entry> ou <form> ?

    # problème position balise manicule ?
    content = content.replace('<REM />', '<g ref="#manicule-glyph"/>')

    # gramGrp
    content = add_lemmaGrp(content)
    
    # sense
    content = add_sense(content)

    # svedet
    content = content.replace(',</svedet>', '</svedet><pc>,</pc>\n').replace('.</svedet>', '</svedet><pc>.</pc>\n')
    content = content.replace('<svedet>', '<form>\n<orth>').replace('</svedet>', '</orth>\n</form>\n')

    # dom
    content = content.replace(',</dom>', '</dom>\n<pc>,</pc>\n').replace('.</dom>', '</dom>\n<pc>.</pc>\n')
    content = content.replace('<dom>', '<usg type="domain">').replace('</dom>', '</usg>')
    # <usg type="domain" ana="#fauconnerie">termes de Fauconnerie</usg>
    
    return content


In [3]:
inputpath = os.path.join('..', 'data', '1743_LeRobert', 'xml')
outputpath = os.path.join('..', 'data', '1743_LeRobert', 'tei')
if not os.path.exists(outputpath):
    os.makedirs(outputpath)

for filename in sorted(os.listdir(inputpath)):
    if not filename.endswith('.xml'):
        continue
    with open(os.path.join(inputpath,filename), 'r', encoding='utf-8') as f:
        content = f.read()

        # suppression de certaines balises (solution temporaire)
        for tag in ['G', 'S', 'I', 'auteur', 'cit', 'oeuvre']:
            content = content.replace(f'<{tag}>', f'').replace(f'</{tag}>', f'')
        
        content = xml_2_tei(content)

        with open(f'{os.path.join(outputpath, filename[:-4])}.tei', 'w', encoding='utf-8') as f:
            f.write(content)

## Check if the XML are valid

In [4]:
inputpath = os.path.join('..', 'data', '1743_LeRobert', 'tei')

for filename in sorted(os.listdir(inputpath)):
    try:
        parser = etree.XMLParser(collect_ids=False, encoding='utf-8') 
        root = etree.parse(os.path.join(inputpath, filename), parser=parser).getroot()    
        #print(root.nsmap)
        print(filename)
        # compte les balise <entry>
        entry_count = len(root.findall('.//entry', namespaces=root.nsmap))
        print(f"Nombre d'entrées : {entry_count}")
        # compte les balise <sense>
        sense_count = len(root.findall('.//sense', namespaces=root.nsmap))
        print(f"Nombre de sens : {sense_count}")
        
    except etree.XMLSyntaxError as e:
        print(f"Erreur de syntaxe XML : {e}")

TR1.tei
Nombre d'entrées : 8244
Nombre de sens : 23449
TR2.tei
Nombre d'entrées : 10146
Nombre de sens : 25844
TR3.tei
Nombre d'entrées : 9818
Nombre de sens : 26609
TR4.tei
Nombre d'entrées : 9470
Nombre de sens : 23490
TR5.tei
Nombre d'entrées : 10702
Nombre de sens : 23129
TR6.tei
Nombre d'entrées : 5586
Nombre de sens : 12483
